In [19]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import tensorflow as tf
from tensorflow.python.platform import tf_logging as logging
from tensorflow.models import slim
from tensorflow.models.slim.preprocessing import inception_preprocessing

from tensorflow.contrib.slim.python.slim.data import tfexample_decoder, dataset
import os, time

"""
Note : Before running the code below ensure you have all the data in TFRecord and appertaining format.
        Else : Follow the code inside TensorFlowIO module to create such files. THis module assumes that 
        the data are in their proper directories
"""
# Code Source : https://kwotsin.github.io/tech/2017/02/11/transfer-learning.html

'\nNote : Before running the code below ensure you have all the data in TFRecord and appertaining format.\n        Else : Follow the code inside TensorFlowIO module to create such files. THis module assumes that \n        the data are in their proper directories\n'

In [15]:
# Directory Listings

# Log file creation
logDir = './log'

# State the image size we are resizing
imageSize = 299 # the default size for inception

# State the path where the label file is stored
labelFile = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/TensorFlowOPS/flowers/labels.txt"
labels = open(labelFile, 'r')

# Dictionary to refer each label to its string name
labelsToNameDict = {}
for record in labels:
    label, stringName = record.split(':')
    stringName = stringName[:-1]
    labelsToNameDict[int(label)] = stringName
    
# Record the file pattern for the TFRecord files
filePattern = "flowers_%s_*.tfRecord"

# A dictionary describing the dataset, to be required by tensorflow dataset Class
itemToDescription = {
    'image': "A 3-channel RGB coloured five different types of flowers, tulips, sunflower, roses, dandelion, daisy",
    'label': "A label value defined to each flower 1: dandelion, 2:roses, 3:sunflower, 4:tulips"
    
}

## Function details:

1. get_splits: It allows us to obtain a specific split form the disk. We store two different types of files in the disk "trai file and validation file". These files were split into two and get_split helps us to ge the required split.

In the process we also collect some information such as number_of samples in the split, and etc.

* Two very important functions are keysTOFeature and itemsToHandlers/
* **keysToFeature**: 

2. DatasetDataProvider: The motive we have here is that we need tensors from 

In [23]:
def getSplits(splitName, datasetDir, filePattern):
    if splitName not in ['train','validation']:
        raise ValueError('The spllit name %s is not recognized, Make sure they are either train, validation or test'%splitName)

    # create a path for data IO: The pattern should be the split name
    filePatternPath = os.path.join(datasetDir, filePattern%(splitName))
    print (filePatternPath)

    # Count the total number of training samples:
    filePattern = filePattern+"%s"%(splitName)
    tfRecordsFilesPath = [os.path.join(datasetDir, file) for file in os.listdir(datasetDir) if file.startswith(filePattern)]
    print (tfRecordsFilesPath)

    # We now count all the records inside the tfRecord files that are separated into multiple shards
    numSamples = 0
    for tfRecord in tfRecordsFilesPath:
        for record in tf.python_io.tf_record_iterator(tfRecord):
            numSamples += 1

    print ('Total records to to be trained are: ', numSamples)

    # Define what type of reader
    reader = tf.TFRecordReader
    
    # Create a keys to feature dictionary
    keysToFeature = {
        'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
        'image/format': tf.FixedLenFeature((), tf.string, default_value='jpg'),
        'image/class/label': tf.FixedLenFeature(
        [], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
    }
    print (keysToFeature)

    # Create the dictionary for the decoder
    itemsToHandlers = {
        'image': tfexample_decoder.Image(),   
        'label': tfexample_decoder.Tensor('image/class/label')
    }
    # tfexample_decoder is the module and Image in the class so basically the key imaage int the dictionary contains the 
    # object of class image as the value.
    # Similarly the key Label also holds the obect of class Tensor in the value

    decoder = tfexample_decoder.TFExampleDecoder(keysToFeature, itemsToHandlers)
    print (decoder)

    labelsTONameDict = labelsToNameDict

    dataetObj = dataset.Dataset(
            data_sources = filePatternPath,
            decoder = decoder,
            reader = reader,
            num_readers = 4,
            num_samples = numSamples,
            num_classes = 5,
            labels_to_name = labelsTONameDict,
            items_to_descriptions = itemToDescription)
    
    return dataetObj

datasetDir = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/TensorFlowOPS/flowers"
split_name = "train"
# filePattern = "flowers_"
filePattern = "flowers_%s_*.tfRecord"

dataset = getSplits(splitName=split_name, datasetDir=datasetDir, filePattern=filePattern)
# 'image': slim.tfexample_decoder.Image(),
#     'label': slim.tfexample_decocer.tensor('image/class/label')

/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/TensorFlowOPS/flowers/flowers_train_*.tfRecord
[]
Total records to to be trained are:  0
{'image/encoded': FixedLenFeature(shape=(), dtype=tf.string, default_value=''), 'image/format': FixedLenFeature(shape=(), dtype=tf.string, default_value='jpg'), 'image/class/label': FixedLenFeature(shape=[], dtype=tf.int64, default_value=<tf.Tensor 'zeros_12:0' shape=() dtype=int64>)}
